In [2]:
import import_ipynb
from helper_functions import is_plugin, has_state

importing Jupyter notebook from helper_functions.ipynb


In [ ]:
def delete_empty_boxes(state, number_of_deleted_boxes):
    if isinstance(state, list):
        new_state = []
        for sub_state in state:
            [new_sub_state, number_of_deleted_boxes] = delete_empty_boxes(sub_state, number_of_deleted_boxes)
            if new_sub_state:
                new_state.append(new_sub_state)
        return [new_state, number_of_deleted_boxes]
    elif isinstance(state, dict):
        if is_plugin(state):
            match state["plugin"]:
                case "box":
                    if is_empty_box(state):
                        number_of_deleted_boxes += 1
                        return [{}, number_of_deleted_boxes]
                case "rows":
                    [state["state"], number_of_deleted_boxes] = delete_empty_boxes(state["state"], number_of_deleted_boxes)
                    if state["state"]:
                        return [state, number_of_deleted_boxes]
                    else:
                        return [{}, number_of_deleted_boxes]
                case _:
                    if has_state(state):
                        [state["state"], number_of_deleted_boxes] = delete_empty_boxes(state["state"], number_of_deleted_boxes)
                    return [state, number_of_deleted_boxes]
    
    return [state, number_of_deleted_boxes]
    

In [1]:
def is_empty_row(row):
    state = row["state"]
    match row["plugin"]:
        case "rows":
            for sub_row in row["state"]:
                if not is_empty_row(sub_row):
                    return False
        case "text":
            return empty_children(state)
        case "image":
            return is_empty_image(state)
        case "equations":
            return is_empty_equation(state)
        case "multimedia":
            return is_empty_multimedia(state)
        case "serloTable":
            #TODO
            return False
        case "highlight":
            #TODO
            return False
        case _:
            raise Exception('Unhandled plugin: {}'.format(row))
    return True

def is_empty_image(image):
    src = image["src"]
    return not src or src.isspace()

def is_empty_multimedia(multimedia):
    if is_empty_row(multimedia["multimedia"]):
        return is_empty_row(multimedia["explanation"]) and is_empty_row(multimedia["multimedia"])

def is_empty_box(box):
    content = box["state"]["content"]
    if content["plugin"] != "rows":
        raise Exception('The box content is invalid: {}'.format(content))
        
    for row in content["state"]:
        if not is_empty_row(row):
            return False
    print("found empty box")
    print(box)
    return True

def is_empty_equation(equation):
    if equation["steps"]:
        return False
    return True


def empty_children(children):
    for child in children:
        if not child:
            continue
        elif "text" in child:
            if child["text"] and not child["text"].isspace():
                return False
        elif "src" in child:
            if child["src"] and not child["src"].isspace():
                return False
        elif "children" in child:
            if not empty_children(child["children"]):
                return False
        else:
            raise Exception('Unhandled text: {}'.format(child))
    return True